# - Data Warehouse -

### Het samenvoegen van bestaande tabellen voor het uiteindelijke 'Datawarehouse'

##### Importeren van benodigde dependencies

In [2]:
import pandas as pd
import pyodbc

import warnings
warnings.filterwarnings("ignore")

# Importeren van de create_connection en run_query functies uit de database_utils.py file
from utils.database_utils import create_connection, run_query

##### 1 - Opbouwen van dataframes voor elke database-tabel

In [36]:
payhistory_df = run_query("SELECT * FROM Production.TransactionHistory", "AdventureWorks2019")
payhistory_archive_df = run_query("SELECT * FROM Production.TransactionHistoryArchive", "AdventureWorks2019")

Je kunt vergelijkbare queries uitvoeren voor andere tabellen die je wilt opnemen.

#### 2 - Data transformatie

In [66]:
create_table_query = """
CREATE TABLE PayHistory (
    IDSK INT NOT NULL,
    TransactionID INT NOT NULL,
    ProductID INT NOT NULL,
    ReferenceOrderID INT NOT NULL,
    ReferenceOrderLineID INT NOT NULL,
    TransactionDate DATETIME NOT NULL,
    TransactionType NVARCHAR(50) NOT NULL,
    Quantity INT NOT NULL,
    ActualCost FLOAT NOT NULL,
    ModifiedDate DATETIME NOT NULL
);
"""

# Create the table in SQL Server
db_name = "AdventureWorks2019"
conn, cursor = create_connection(db_name)
cursor.execute(create_table_query)
conn.commit()
conn.close()


In [37]:
payhistory_archive_df['IDSK'] = range(1, len(payhistory_archive_df) + 1)
payhistory_df['IDSK'] = range(1, len(payhistory_df) + 1)

Na het opbouwen van de dataframes voor elke database-tabel, kunnen we beginnen met het transformeren van de data. Dit omvat het samenvoegen van tabellen, het toepassen van filters, het uitvoeren van berekeningen, enzovoort.

In [47]:
# Concatenate the dataframes
combined_df = pd.concat([payhistory_df, payhistory_archive_df], ignore_index=True)

In [55]:
combined_df.dtypes

TransactionID                    int64
ProductID                        int64
ReferenceOrderID                 int64
ReferenceOrderLineID             int64
TransactionDate         datetime64[ns]
TransactionType                 object
Quantity                         int64
ActualCost                     float64
ModifiedDate            datetime64[ns]
IDSK                             int64
dtype: object

#### 3 - Data loading

Na het transformeren van de data, kunnen we de resulterende dataframe in de doeltabel van ons datawarehouse laden.

In [67]:
# Verbinding maken met de database
test_database_name = 'AdventureWorks2019'
conn, cursor = create_connection(test_database_name)

sql_template = """
INSERT INTO PayHistory (
    IDSK, TransactionID, ProductID, ReferenceOrderID, ReferenceOrderLineID,
    TransactionDate, TransactionType, Quantity, ActualCost, ModifiedDate
) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

# Itereer over de dataframe en voer de inserties uit
for index, row in combined_df.iterrows():
    row['TransactionDate'] = row['TransactionDate'].strftime('%Y-%m-%d %H:%M:%S')
    row['ModifiedDate'] = row['ModifiedDate'].strftime('%Y-%m-%d %H:%M:%S')

    # Tuple met waarden voor de invoegquery
    values = (
        row['IDSK'], row['TransactionID'], row['ProductID'], row['ReferenceOrderID'], row['ReferenceOrderLineID'],
        row['TransactionDate'], row['TransactionType'], row['Quantity'], row['ActualCost'],
        row['ModifiedDate']
    )
    # Uitvoeren van de query
    cursor.execute(sql_template, values)

# Commit de transacties en sluit de verbinding
conn.commit()
conn.close()

**Note:** Voeg indien nodig zoveel Markdown- of codeblokken toe als nodig is.

#### 4 -  Data Quality Checks

Voeg controles toe om de kwaliteit van de gegevens te waarborgen voordat ze worden geladen in het datawarehouse:

In [49]:
# Controleren op ontbrekende waarden
missing_values = combined_df.isnull().sum()

# Controleren op duplicaten
duplicate_rows = combined_df.duplicated().sum()

# Weergave van resultaten
print("Aantal ontbrekende waarden:", missing_values)
print("Aantal duplicaten:", duplicate_rows)

Aantal ontbrekende waarden: TransactionID           0
ProductID               0
ReferenceOrderID        0
ReferenceOrderLineID    0
TransactionDate         0
TransactionType         0
Quantity                0
ActualCost              0
ModifiedDate            0
IDSK                    0
dtype: int64
Aantal duplicaten: 0


**Note:** Dit is optioneel, het leek mij opzich best handig om te doen.